# ANÁLISE AVANÇADA DE ABSENTISMO

**Framework Analítico Completo**: Da Descrição à Prescrição

---

## Estrutura da Análise

1. **Preparação e Limpeza de Dados**
2. **Descrição Fundamental dos Dados**
3. **Conceito de Spells (Episódios de Ausência)**
4. **Métricas Core (KPIs Essenciais)**
5. **Bradford Factor Analysis**
6. **Deteção de Padrões Suspeitos**
7. **Análise de Cohorts (por Data de Ingresso)**
8. **Clustering de Perfis de Absentismo**
9. **Network Analysis (Coincidências)**
10. **Event Detection & Anomaly Detection**
11. **Visualizações Avançadas**
12. **Síntese Executiva e Ações Recomendadas**

---

## 1. PREPARAÇÃO E LIMPEZA DE DADOS

In [ ]:
# Imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print('✓ Bibliotecas carregadas')

In [ ]:
# 1.1 Carregar dados
print('Carregando dados...')
df_raw = pd.read_csv('combined_data.csv')
df_raw['Data'] = pd.to_datetime(df_raw['Data'])

print(f'   Registos totais: {len(df_raw):,}')
print(f'   Colaboradores únicos: {df_raw["login_colaborador"].nunique():,}')
print(f'   Período: {df_raw["Data"].min().date()} até {df_raw["Data"].max().date()}')

# Carregar NOVOS códigos (V2)
print('\nCarregando nova classificação (Códigos_V2)...')
df_codigos = pd.read_excel('códigos_V2.xlsx')
print(f'   Códigos carregados: {len(df_codigos)}')
print(f'\nNivel 1 categories: {df_codigos["Nivel 1"].nunique()}')
print(df_codigos['Nivel 1'].value_counts())
print(f'\nNivel 2 categories: {df_codigos["Nivel 2"].nunique()}')
print(df_codigos['Nivel 2'].value_counts())

In [ ]:
# 1.2 Merge com novos códigos
print('Aplicando nova classificação aos dados...')

# Merge
df_raw = df_raw.merge(
    df_codigos,
    left_on='segmento_processado_codigo',
    right_on='Codigo Segmento',
    how='left'
)

# Verificar códigos não mapeados
codigos_sem_match = df_raw[df_raw['Nivel 1'].isna()]['segmento_processado_codigo'].unique()
if len(codigos_sem_match) > 0:
    print(f'\n⚠️  ATENÇÃO: {len(codigos_sem_match)} códigos sem correspondência:')
    print(codigos_sem_match)
    print(f'   Registos afetados: {df_raw["Nivel 1"].isna().sum():,}')
else:
    print('\n✓ Todos os códigos mapeados com sucesso')

print(f'\nTotal de registos: {len(df_raw):,}')

### 1.3 Identificar e Remover Incompatibilidades

**Particularidade dos dados**: Num mesmo dia, um colaborador pode ter **múltiplos registos**.

**Regras de Compatibilidade** (Nivel 2):
- ✅ **Atraso** + Presença
- ✅ **Atraso** + Exame Escolar  
- ✅ **Presença** + Exame Escolar
- ❌ **Tudo o resto é INCOMPATÍVEL**

Vamos criar uma matriz de compatibilidade e identificar dias problemáticos.

In [ ]:
# 1.3.1 Criar matriz de compatibilidade (Nivel 2)
print('Criando matriz de compatibilidade (Nivel 2)...')

# Lista de categorias Nivel 2
lista_nivel2 = sorted(df_codigos['Nivel 2'].unique())
print(f'\nCategorias Nivel 2: {len(lista_nivel2)}')
for cat in lista_nivel2:
    print(f'  - {cat}')

# Criar matriz (default: INCOMPATÍVEL)
matriz_compat = pd.DataFrame('Incompatível', index=lista_nivel2, columns=lista_nivel2)

# REGRA 1: Categoria consigo mesma = COMPATÍVEL
for cat in lista_nivel2:
    matriz_compat.loc[cat, cat] = 'Compatível'

# REGRA 2: Apenas 3 pares compatíveis
pares_compativeis = [
    ('Atraso', 'Presença'),
    ('Atraso', 'Exame Escolar'),
    ('Presença', 'Exame Escolar')
]

for cat1, cat2 in pares_compativeis:
    if cat1 in matriz_compat.index and cat2 in matriz_compat.columns:
        matriz_compat.loc[cat1, cat2] = 'Compatível'
        matriz_compat.loc[cat2, cat1] = 'Compatível'  # Simétrica

# Exportar matriz para validação
with pd.ExcelWriter('matriz_compatibilidade_nivel2_v2.xlsx') as writer:
    matriz_compat.to_excel(writer, sheet_name='Matriz')
    
print('\n✓ Matriz de compatibilidade criada e exportada')
print(f'\nPares COMPATÍVEIS (além da diagonal):' )
for cat1, cat2 in pares_compativeis:
    print(f'  ✓ {cat1} + {cat2}')

In [ ]:
# 1.3.2 Identificar dias com múltiplos registos
print('Identificando dias com múltiplos registos...')

registros_por_dia = df_raw.groupby(['login_colaborador', 'Data']).size()
dias_duplicados = registros_por_dia[registros_por_dia > 1]

print(f'   Dias com múltiplos registos: {len(dias_duplicados):,}')
print(f'   ({len(dias_duplicados) / len(df_raw.groupby(["login_colaborador", "Data"])) * 100:.2f}% do total de dias-colaborador)')

In [ ]:
# 1.3.3 Testar incompatibilidades (VERSÃO OTIMIZADA)
print('Testando incompatibilidades nos dias duplicados...')

# PRÉ-FILTRAR apenas dias duplicados
dias_dup_list = dias_duplicados.index.tolist()
df_dup = df_raw.set_index(['login_colaborador', 'Data']).loc[dias_dup_list].reset_index()

print(f'   Registos a testar: {len(df_dup):,}')

# PRÉ-AGRUPAR dados (UMA vez!)
df_dup_grouped = df_dup.groupby(['login_colaborador', 'Data']).apply(
    lambda g: pd.Series({
        'categorias_nivel2': list(g['Nivel 2'].dropna().unique()),
        'codigos': list(g['segmento_processado_codigo'].unique()),
        'nome': g['nome_colaborador'].iloc[0]
    })
).reset_index()

print(f'   Dias únicos a testar: {len(df_dup_grouped):,}')

# Identificar incompatibilidades
print('\nTestando pares de categorias...')
dias_incompativeis = []

for idx, row in df_dup_grouped.iterrows():
    if idx % 5000 == 0 and idx > 0:
        print(f'   Processados {idx:,}/{len(df_dup_grouped):,} dias...')
    
    login = row['login_colaborador']
    data = row['Data']
    categorias = row['categorias_nivel2']
    codigos = row['codigos']
    nome = row['nome']
    
    # Testar todos os pares
    incompativel_encontrado = False
    pares_incompativeis = []
    
    for i, cat1 in enumerate(categorias):
        for cat2 in categorias[i+1:]:
            if cat1 in matriz_compat.index and cat2 in matriz_compat.columns:
                if matriz_compat.loc[cat1, cat2] == 'Incompatível':
                    incompativel_encontrado = True
                    pares_incompativeis.append(f'{cat1} + {cat2}')
    
    if incompativel_encontrado:
        dias_incompativeis.append({
            'login_colaborador': login,
            'Data': data,
            'nome_colaborador': nome,
            'categorias': ', '.join(categorias),
            'codigos': ', '.join(codigos),
            'pares_incompativeis': ' | '.join(pares_incompativeis)
        })

df_incompativeis = pd.DataFrame(dias_incompativeis)

print(f'\n✓ Teste concluído')
print(f'\n🔴 INCOMPATIBILIDADES ENCONTRADAS: {len(df_incompativeis)}')

if len(df_incompativeis) > 0:
    print(f'\nDistribuição por par incompatível:')
    for par in df_incompativeis['pares_incompativeis'].str.split(' | ').explode().value_counts().head(10).items():
        print(f'   {par[0]}: {par[1]} casos')
    
    # Exportar para Excel
    df_incompativeis.to_excel('incompatibilidades_encontradas_v2.xlsx', index=False)
    print('\n✓ Detalhes exportados para: incompatibilidades_encontradas_v2.xlsx')

In [ ]:
# 1.3.4 Remover dias incompatíveis (VERSÃO VETORIZADA)
print('Removendo dias incompatíveis...')

n_antes = len(df_raw)

if len(df_incompativeis) > 0:
    # Criar DataFrame dos dias a remover
    dias_remover = df_incompativeis[['login_colaborador', 'Data']].copy()
    
    # Merge com indicador
    df_temp = df_raw.merge(dias_remover, on=['login_colaborador', 'Data'], how='left', indicator=True)
    
    # Manter apenas linhas NÃO marcadas
    df_limpo = df_temp[df_temp['_merge'] == 'left_only'].drop('_merge', axis=1)
else:
    df_limpo = df_raw.copy()
    print('   Nenhuma incompatibilidade encontrada')

n_depois = len(df_limpo)
print(f'\n✓ Registos removidos: {n_antes - n_depois:,}')
print(f'   ({(n_antes - n_depois) / n_antes * 100:.3f}% do total)')
print(f'\nDataset limpo: {n_depois:,} registos')

In [ ]:
# 1.3.5 Agregação final: 1 linha por dia-colaborador
print('Agregando dados: 1 linha por dia-colaborador...')

# Definir regras de agregação
agg_rules = {
    'nome_colaborador': 'first',
    'categoria_profissional': 'first',
    'segmento_processado_codigo': lambda x: ', '.join(x.unique()),  # Concatenar se múltiplos
    'Nivel 1': lambda x: ', '.join(x.dropna().unique()),
    'Nivel 2': lambda x: ', '.join(x.dropna().unique()),
}

# Adicionar operação se existir
if 'operacao' in df_limpo.columns:
    agg_rules['operacao'] = 'first'

df = df_limpo.groupby(['login_colaborador', 'Data']).agg(agg_rules).reset_index()

print(f'\n✓ Dataset agregado: {len(df):,} dias-colaborador')
print(f'   Colaboradores únicos: {df["login_colaborador"].nunique():,}')
print(f'   Período: {df["Data"].min().date()} até {df["Data"].max().date()}')

---

## 2. DESCRIÇÃO FUNDAMENTAL DOS DADOS

In [ ]:
# 2.1 Estatísticas descritivas
print('=== DESCRIÇÃO DO DATASET ===')
print(f'\n📊 DIMENSÃO')
print(f'   Total de registos (dias-colaborador): {len(df):,}')
print(f'   Colaboradores únicos: {df["login_colaborador"].nunique():,}')
print(f'   Período: {df["Data"].min().date()} até {df["Data"].max().date()}')
print(f'   Dias calendário: {(df["Data"].max() - df["Data"].min()).days + 1}')

print(f'\n📋 DISTRIBUIÇÃO POR NIVEL 1')
dist_nivel1 = df['Nivel 1'].value_counts()
for cat, count in dist_nivel1.items():
    pct = count / len(df) * 100
    print(f'   {cat:25s}: {count:7,} ({pct:5.2f}%)')

print(f'\n📋 DISTRIBUIÇÃO POR NIVEL 2')
dist_nivel2 = df['Nivel 2'].value_counts()
for cat, count in dist_nivel2.head(15).items():
    pct = count / len(df) * 100
    print(f'   {cat:30s}: {count:7,} ({pct:5.2f}%)')

if 'operacao' in df.columns:
    print(f'\n🏢 OPERAÇÕES')
    print(f'   Operações únicas: {df["operacao"].nunique()}')
    print(f'   Top 10 operações:')
    for op, count in df['operacao'].value_counts().head(10).items():
        pct = count / len(df) * 100
        print(f'      {op:40s}: {count:6,} ({pct:5.2f}%)')

print(f'\n👥 CATEGORIAS PROFISSIONAIS')
print(f'   Categorias únicas: {df["categoria_profissional"].nunique()}')
for cat, count in df['categoria_profissional'].value_counts().head(10).items():
    pct = count / len(df) * 100
    print(f'   {cat:30s}: {count:6,} ({pct:5.2f}%)')

In [ ]:
# 2.2 Visualização: Distribuição temporal
print('Criando visualização de distribuição temporal...')

# Agrupar por mês
df['Ano_Mes'] = df['Data'].dt.to_period('M').astype(str)
df_mensal = df.groupby('Ano_Mes').size().reset_index(name='Registos')

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_mensal['Ano_Mes'],
    y=df_mensal['Registos'],
    mode='lines+markers',
    name='Registos',
    line=dict(color='#1f77b4', width=2),
    marker=dict(size=6)
))

fig.update_layout(
    title='Evolução Temporal dos Registos',
    xaxis_title='Mês',
    yaxis_title='Número de Registos',
    height=400,
    hovermode='x unified'
)

fig.show()
print('✓ Visualização criada')

---

## 3. CONCEITO DE SPELLS (EPISÓDIOS DE AUSÊNCIA)

**Spell** = Episódio contínuo de ausência (um ou mais dias consecutivos).

**Importância**:
- 1 spell de 5 dias vs 5 spells de 1 dia = **impacto operacional MUITO diferente**
- Permite calcular métricas sofisticadas:
  - **Frequency Rate**: Taxa de spells por colaborador
  - **Mean Spell Duration**: Duração média de cada episódio
  - **Short-term spells**: ≤ 3 dias (possível "abuso")
  - **Long-term spells**: > 14 dias (doença grave, tratamento médico)

**Algoritmo**:
1. Ordenar registos por colaborador e data
2. Identificar quebras de continuidade (gap > 1 dia)
3. Agrupar dias consecutivos no mesmo spell
4. Calcular duração, tipo predominante, etc.

In [ ]:
# 3.1 Criar spells de AUSÊNCIA (excluir Presenças e Atrasos)
print('Criando spells de ausência...')

# Filtrar apenas AUSÊNCIAS (excluir Trabalho Pago, Atraso)
df_ausencias = df[df['Nivel 1'].isin(['Falta Justificada', 'Falta Injustificada', 'Ausência'])].copy()

print(f'   Registos de ausência: {len(df_ausencias):,}')
print(f'   Colaboradores com ausências: {df_ausencias["login_colaborador"].nunique():,}')

# Ordenar por colaborador e data
df_ausencias = df_ausencias.sort_values(['login_colaborador', 'Data']).reset_index(drop=True)

# Calcular diferença de dias entre registos consecutivos
df_ausencias['dias_desde_anterior'] = df_ausencias.groupby('login_colaborador')['Data'].diff().dt.days

# Novo spell quando:
# 1. Primeiro registo do colaborador (dias_desde_anterior = NaN)
# 2. Gap > 1 dia (não consecutivo)
df_ausencias['novo_spell'] = (
    (df_ausencias['dias_desde_anterior'].isna()) |  # Primeiro registo
    (df_ausencias['dias_desde_anterior'] > 1)        # Gap de dias
)

# Atribuir ID único a cada spell
df_ausencias['spell_id'] = df_ausencias['novo_spell'].cumsum()

print(f'\n✓ Spells identificados: {df_ausencias["spell_id"].nunique():,}')

In [ ]:
# 3.2 Agregar informação por spell
print('Agregando informação por spell...')

df_spells = df_ausencias.groupby('spell_id').agg({
    'login_colaborador': 'first',
    'nome_colaborador': 'first',
    'categoria_profissional': 'first',
    'Data': ['min', 'max', 'count'],  # Início, fim, duração
    'Nivel 1': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],  # Tipo predominante
    'Nivel 2': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
}).reset_index()

# Renomear colunas
df_spells.columns = [
    'spell_id', 'login_colaborador', 'nome_colaborador', 'categoria_profissional',
    'data_inicio', 'data_fim', 'duracao_dias',
    'nivel1_predominante', 'nivel2_predominante'
]

# Adicionar operação se existir
if 'operacao' in df_ausencias.columns:
    df_spells = df_spells.merge(
        df_ausencias.groupby('spell_id')['operacao'].first().reset_index(),
        on='spell_id'
    )

# Adicionar features
df_spells['dia_semana_inicio'] = df_spells['data_inicio'].dt.day_name()
df_spells['dia_semana_fim'] = df_spells['data_fim'].dt.day_name()
df_spells['mes'] = df_spells['data_inicio'].dt.month
df_spells['ano'] = df_spells['data_inicio'].dt.year

# Categorizar spells
df_spells['categoria_spell'] = pd.cut(
    df_spells['duracao_dias'],
    bins=[0, 1, 3, 7, 14, float('inf')],
    labels=['1 dia', '2-3 dias', '4-7 dias', '8-14 dias', '>14 dias']
)

df_spells['short_term'] = df_spells['duracao_dias'] <= 3
df_spells['long_term'] = df_spells['duracao_dias'] > 14

print(f'\n✓ Dataset de spells criado: {len(df_spells):,} spells')
print(f'\nDistribuição por duração:')
print(df_spells['categoria_spell'].value_counts().sort_index())

print(f'\nEstatísticas de duração:')
print(f'   Média: {df_spells["duracao_dias"].mean():.2f} dias')
print(f'   Mediana: {df_spells["duracao_dias"].median():.0f} dias')
print(f'   P75: {df_spells["duracao_dias"].quantile(0.75):.0f} dias')
print(f'   P95: {df_spells["duracao_dias"].quantile(0.95):.0f} dias')
print(f'   Máximo: {df_spells["duracao_dias"].max()} dias')

In [ ]:
# 3.3 Análise de spells por colaborador
print('Analisando spells por colaborador...')

df_colab_spells = df_spells.groupby('login_colaborador').agg({
    'spell_id': 'count',  # Frequency rate (número de spells)
    'duracao_dias': ['sum', 'mean', 'median', 'std'],
    'short_term': 'sum',  # Número de spells curtos
    'long_term': 'sum',   # Número de spells longos
}).reset_index()

df_colab_spells.columns = [
    'login_colaborador', 'num_spells', 
    'total_dias_ausentes', 'mean_spell_duration', 'median_spell_duration', 'std_spell_duration',
    'num_short_term_spells', 'num_long_term_spells'
]

# Adicionar nome
df_colab_spells = df_colab_spells.merge(
    df[['login_colaborador', 'nome_colaborador']].drop_duplicates(),
    on='login_colaborador'
)

print(f'\n✓ Análise por colaborador criada: {len(df_colab_spells):,} colaboradores')
print(f'\nTop 10 colaboradores por número de spells:')
print(df_colab_spells.nlargest(10, 'num_spells')[[
    'nome_colaborador', 'num_spells', 'total_dias_ausentes', 'mean_spell_duration'
]].to_string(index=False))

---

## 4. MÉTRICAS CORE (KPIs ESSENCIAIS)

Baseadas em frameworks de HR Analytics (AIHR, Fitzgerald HR).

In [ ]:
# 4.1 Calcular métricas fundamentais
print('=== MÉTRICAS CORE ===')

# Período de análise
data_inicio = df['Data'].min()
data_fim = df['Data'].max()
dias_calendario = (data_fim - data_inicio).days + 1
num_colaboradores = df['login_colaborador'].nunique()

# Contar registos por tipo
num_presencas = df[df['Nivel 1'] == 'Trabalho Pago'].shape[0]
num_atrasos = df[df['Nivel 1'] == 'Atraso'].shape[0]
num_faltas = df[df['Nivel 1'].isin(['Falta Justificada', 'Falta Injustificada'])].shape[0]
num_ausencias_medicas = df[df['Nivel 2'] == 'Ausência Médica'].shape[0]

# KPI 1: Taxa de Absentismo Global
# % Absentismo = Total de Faltas / (Presenças + Total de Faltas)
taxa_absentismo_global = (num_faltas / (num_presencas + num_faltas)) * 100

# KPI 2: Lost Time Rate (dias perdidos por FTE)
total_dias_perdidos = df_spells['duracao_dias'].sum()
lost_time_rate = total_dias_perdidos / num_colaboradores

# KPI 3: Frequency Rate (spells por colaborador)
frequency_rate = len(df_spells) / num_colaboradores

# KPI 4: Mean Spell Duration
mean_spell_duration = df_spells['duracao_dias'].mean()

# KPI 5: Taxa de Atrasos
# % Atrasos = Atrasos / Total Presenças
taxa_atrasos = (num_atrasos / num_presencas) * 100 if num_presencas > 0 else 0

# KPI 6: Taxa de Zero Ausências
colaboradores_sem_ausencias = num_colaboradores - df_spells['login_colaborador'].nunique()
taxa_zero_ausencias = (colaboradores_sem_ausencias / num_colaboradores) * 100

# Exibir resultados
print(f'\n📊 PERÍODO DE ANÁLISE')
print(f'   {data_inicio.date()} até {data_fim.date()} ({dias_calendario} dias)')
print(f'   Colaboradores únicos: {num_colaboradores:,}')

print(f'\n🎯 KPIs PRINCIPAIS')
print(f'   Taxa de Absentismo Global: {taxa_absentismo_global:.2f}%')
print(f'   Lost Time Rate: {lost_time_rate:.2f} dias/colaborador')
print(f'   Frequency Rate: {frequency_rate:.2f} spells/colaborador')
print(f'   Mean Spell Duration: {mean_spell_duration:.2f} dias')
print(f'   Taxa de Atrasos: {taxa_atrasos:.2f}%')
print(f'   Taxa de Zero Ausências: {taxa_zero_ausencias:.2f}%')

print(f'\n📈 BREAKDOWN')
print(f'   Presenças: {num_presencas:,}')
print(f'   Atrasos: {num_atrasos:,}')
print(f'   Faltas (Total): {num_faltas:,}')
print(f'      - Justificadas: {df[df["Nivel 1"] == "Falta Justificada"].shape[0]:,}')
print(f'      - Injustificadas: {df[df["Nivel 1"] == "Falta Injustificada"].shape[0]:,}')
print(f'   Ausências Médicas: {num_ausencias_medicas:,}')
print(f'\n   Total de Spells: {len(df_spells):,}')
print(f'      - Short-term (≤3 dias): {df_spells["short_term"].sum():,}')
print(f'      - Long-term (>14 dias): {df_spells["long_term"].sum():,}')

---

## 5. BRADFORD FACTOR ANALYSIS

**Bradford Factor** = S² × D
- S = Número de spells (episódios)
- D = Total de dias ausentes

**Princípio**: Ausências curtas e frequentes são **mais disruptivas** que ausências longas ocasionais.

**Exemplo**:
- Colaborador A: 1 spell de 10 dias → B = 1² × 10 = **10**
- Colaborador B: 10 spells de 1 dia → B = 10² × 10 = **1000** (100x pior!)

**Thresholds típicos** (Call Centre Helper):
- < 45: Aceitável
- 45-100: Conversa informal  
- 100-200: Revisão formal
- 200-500: Aviso escrito
- 500-900: Ação disciplinar
- \> 900: Preocupação séria

In [ ]:
# 5.1 Calcular Bradford Factor por colaborador
print('Calculando Bradford Factor...')

df_bradford = df_colab_spells.copy()
df_bradford['bradford_score'] = (df_bradford['num_spells'] ** 2) * df_bradford['total_dias_ausentes']

# Categorizar por risk level
def categorizar_bradford(score):
    if score < 45:
        return '1. Aceitável (<45)'
    elif score < 100:
        return '2. Conversa Informal (45-100)'
    elif score < 200:
        return '3. Revisão Formal (100-200)'
    elif score < 500:
        return '4. Aviso Escrito (200-500)'
    elif score < 900:
        return '5. Ação Disciplinar (500-900)'
    else:
        return '6. Preocupação Séria (>900)'

df_bradford['risk_level'] = df_bradford['bradford_score'].apply(categorizar_bradford)

print(f'\n✓ Bradford Factor calculado para {len(df_bradford):,} colaboradores')
print(f'\nDistribuição por Risk Level:')
dist_risk = df_bradford['risk_level'].value_counts().sort_index()
for level, count in dist_risk.items():
    pct = count / len(df_bradford) * 100
    print(f'   {level:40s}: {count:4,} ({pct:5.2f}%)')

print(f'\nEstatísticas do Bradford Score:')
print(f'   Média: {df_bradford["bradford_score"].mean():.2f}')
print(f'   Mediana: {df_bradford["bradford_score"].median():.2f}')
print(f'   P75: {df_bradford["bradford_score"].quantile(0.75):.2f}')
print(f'   P90: {df_bradford["bradford_score"].quantile(0.90):.2f}')
print(f'   P95: {df_bradford["bradford_score"].quantile(0.95):.2f}')
print(f'   Máximo: {df_bradford["bradford_score"].max():.2f}')

In [ ]:
# 5.2 Top colaboradores para atenção de RH
print('\n🔴 TOP 20 COLABORADORES PARA ATENÇÃO DE RH (Bradford Factor)')
print('='*100)

top20_bradford = df_bradford.nlargest(20, 'bradford_score')[
    ['nome_colaborador', 'num_spells', 'total_dias_ausentes', 
     'mean_spell_duration', 'bradford_score', 'risk_level']
].copy()

top20_bradford.columns = ['Nome', 'Nº Spells', 'Total Dias', 'Duração Média', 'Bradford', 'Risk Level']

print(top20_bradford.to_string(index=False))

# Exportar para Excel
top20_bradford.to_excel('bradford_factor_top20.xlsx', index=False)
print('\n✓ Top 20 exportado para: bradford_factor_top20.xlsx')

In [ ]:
# 5.3 Visualização: Distribuição de Bradford Factor
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Distribuição por Risk Level', 'Scatter: Spells vs Dias (Bradford)']
)

# Gráfico 1: Barras de risk level
dist_risk_sorted = dist_risk.sort_index()
fig.add_trace(
    go.Bar(
        x=[x.split('. ')[1].split(' (')[0] for x in dist_risk_sorted.index],
        y=dist_risk_sorted.values,
        marker_color=['green', 'yellow', 'orange', 'red', 'darkred', 'black'][:len(dist_risk_sorted)],
        text=dist_risk_sorted.values,
        textposition='auto'
    ),
    row=1, col=1
)

# Gráfico 2: Scatter plot
# Colorir por risk level
color_map = {
    '1. Aceitável (<45)': 'green',
    '2. Conversa Informal (45-100)': 'yellow',
    '3. Revisão Formal (100-200)': 'orange',
    '4. Aviso Escrito (200-500)': 'red',
    '5. Ação Disciplinar (500-900)': 'darkred',
    '6. Preocupação Séria (>900)': 'black'
}

for risk_level in df_bradford['risk_level'].unique():
    df_temp = df_bradford[df_bradford['risk_level'] == risk_level]
    fig.add_trace(
        go.Scatter(
            x=df_temp['num_spells'],
            y=df_temp['total_dias_ausentes'],
            mode='markers',
            name=risk_level.split('. ')[1].split(' (')[0],
            marker=dict(
                size=8,
                color=color_map.get(risk_level, 'gray'),
                opacity=0.7
            ),
            text=df_temp['nome_colaborador'],
            hovertemplate='<b>%{text}</b><br>Spells: %{x}<br>Dias: %{y}<extra></extra>'
        ),
        row=1, col=2
    )

fig.update_xaxes(title_text='Risk Level', row=1, col=1)
fig.update_yaxes(title_text='Número de Colaboradores', row=1, col=1)
fig.update_xaxes(title_text='Número de Spells', row=1, col=2)
fig.update_yaxes(title_text='Total Dias Ausentes', row=1, col=2)

fig.update_layout(
    title='Bradford Factor Analysis',
    height=500,
    showlegend=True
)

fig.show()

In [ ]:
# 5.4 Visualização EXTRA: Bradford Heatmap por Operação e Categoria
print('\nCriando heatmap Bradford por Operação × Categoria...')

if 'operacao' in df.columns:
    # Merge Bradford scores com operação
    df_bradford_op = df_bradford.merge(
        df[['login_colaborador', 'operacao']].drop_duplicates(),
        on='login_colaborador',
        how='left'
    )

    # Merge com categoria profissional
    df_bradford_op = df_bradford_op.merge(
        df[['login_colaborador', 'categoria_profissional']].drop_duplicates(),
        on='login_colaborador',
        how='left'
    )

    # Top 10 operações
    top_ops = df['operacao'].value_counts().head(10).index
    df_bradford_op_filt = df_bradford_op[df_bradford_op['operacao'].isin(top_ops)]

    # Top 5 categorias
    top_cats = df['categoria_profissional'].value_counts().head(5).index
    df_bradford_op_filt = df_bradford_op_filt[df_bradford_op_filt['categoria_profissional'].isin(top_cats)]

    # Pivot: operação × categoria
    pivot_bradford = df_bradford_op_filt.pivot_table(
        index='operacao',
        columns='categoria_profissional',
        values='bradford_score',
        aggfunc='mean'
    ).fillna(0)

    # Heatmap
    fig = go.Figure(data=go.Heatmap(
        z=pivot_bradford.values,
        x=pivot_bradford.columns,
        y=pivot_bradford.index,
        colorscale='RdYlGn_r',  # Vermelho = alto, Verde = baixo
        text=pivot_bradford.values.round(0),
        texttemplate='%{text}',
        textfont={"size": 10},
        hovertemplate='Operação: %{y}<br>Categoria: %{x}<br>Bradford Médio: %{z:.0f}<extra></extra>'
    ))

    fig.update_layout(
        title='Bradford Factor Médio: Operação × Categoria Profissional',
        xaxis_title='Categoria Profissional',
        yaxis_title='Operação',
        height=600,
        width=1000
    )

    fig.show()
    print('✓ Heatmap criado')
else:
    print('⚠️ Campo "operacao" não encontrado')

In [ ]:
# 5.5 Visualização EXTRA: Funil de Ação (Bradford Thresholds)
print('\nCriando funil de ação...')

# Contar colaboradores por threshold
thresholds = [
    (0, 45, 'Aceitável'),
    (45, 100, 'Conversa Informal'),
    (100, 200, 'Revisão Formal'),
    (200, 500, 'Aviso Escrito'),
    (500, 900, 'Ação Disciplinar'),
    (900, float('inf'), 'Preocupação Séria')
]

funnel_data = []
for min_val, max_val, label in thresholds:
    count = ((df_bradford['bradford_score'] >= min_val) &
             (df_bradford['bradford_score'] < max_val)).sum()
    funnel_data.append({'Nível': label, 'Colaboradores': count})

df_funnel = pd.DataFrame(funnel_data)

# Criar funil (inverted para mostrar prioridade)
fig = go.Figure()

colors = ['green', 'yellow', 'orange', 'red', 'darkred', 'black']

fig.add_trace(go.Funnel(
    y=df_funnel['Nível'],
    x=df_funnel['Colaboradores'],
    textposition='inside',
    textinfo='value+percent initial',
    marker=dict(color=colors),
    connector={"line": {"color": "gray", "dash": "dot", "width": 2}}
))

fig.update_layout(
    title='Funil de Ação: Colaboradores por Nível de Risco (Bradford Factor)',
    height=500,
    width=800
)

fig.show()

print('\n📊 FUNIL DE AÇÃO')
for _, row in df_funnel.iterrows():
    print(f'   {row["Nível"]:30s}: {row["Colaboradores"]:4,} colaboradores')

In [ ]:
# 5.6 Visualização EXTRA: Timeline de Spells (Frequência vs Duração)
print('\nCriando timeline de spells...')

# Agrupar spells por mês
df_spells['mes_inicio'] = df_spells['data_inicio'].dt.to_period('M').astype(str)

spells_mensal = df_spells.groupby('mes_inicio').agg({
    'spell_id': 'count',
    'duracao_dias': 'mean'
}).reset_index()

spells_mensal.columns = ['Mes', 'Num_Spells', 'Duracao_Media']

# Dual-axis plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        x=spells_mensal['Mes'],
        y=spells_mensal['Num_Spells'],
        name='Número de Spells',
        marker_color='lightblue'
    ),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=spells_mensal['Mes'],
        y=spells_mensal['Duracao_Media'],
        name='Duração Média (dias)',
        mode='lines+markers',
        line=dict(color='red', width=3),
        marker=dict(size=8)
    ),
    secondary_y=True
)

fig.update_xaxes(title_text='Mês')
fig.update_yaxes(title_text='Número de Spells', secondary_y=False)
fig.update_yaxes(title_text='Duração Média (dias)', secondary_y=True)

fig.update_layout(
    title='Evolução Temporal: Frequência vs Duração de Spells',
    height=500,
    hovermode='x unified'
)

fig.show()
print('✓ Timeline criada')

---

**CONTINUA NA PRÓXIMA CÉLULA**:
- Deteção de Padrões Suspeitos
- Análise de Cohorts
- Clustering
- Network Analysis
- Event Detection
- Visualizações Avançadas

---

## 6. DETEÇÃO DE PADRÕES SUSPEITOS 🔍

**Red Flags** para possível abuso ou comportamento anómalo:
- **Weekend Pattern**: Faltas consistentes às sextas/segundas
- **Bridge Pattern**: Ausências adjacentes a feriados  
- **Temporal Consistency**: Sempre no mesmo dia/semana/mês
- **Fragmentação**: Bradford score alto com poucos dias totais

In [ ]:
# 6.1 Padrão Segunda/Sexta - Spells que começam/terminam em fim de semana
print('Analisando padrões de segunda/sexta...')

# Proporção de spells que começam à segunda
spells_inicio_segunda = df_spells[df_spells['dia_semana_inicio'] == 'Monday'].shape[0]
prop_inicio_segunda = spells_inicio_segunda / len(df_spells) * 100

# Proporção de spells que terminam à sexta
spells_fim_sexta = df_spells[df_spells['dia_semana_fim'] == 'Friday'].shape[0]
prop_fim_sexta = spells_fim_sexta / len(df_spells) * 100

# Spells que combinam ambos (ponte de fim de semana)
spells_ponte_fds = df_spells[
    (df_spells['dia_semana_inicio'] == 'Monday') & 
    (df_spells['dia_semana_fim'] == 'Friday')
].shape[0]

print(f'\n📊 PADRÃO SEGUNDA/SEXTA')
print(f'   Spells que começam à segunda: {spells_inicio_segunda:,} ({prop_inicio_segunda:.2f}%)')
print(f'   Spells que terminam à sexta: {spells_fim_sexta:,} ({prop_fim_sexta:.2f}%)')
print(f'   Spells "ponte de semana" (2ª → 6ª): {spells_ponte_fds:,}')

# Análise por colaborador
df_padroes_colab = df_spells.groupby('login_colaborador').agg({
    'spell_id': 'count',
    'dia_semana_inicio': lambda x: (x == 'Monday').sum(),
    'dia_semana_fim': lambda x: (x == 'Friday').sum()
}).reset_index()

df_padroes_colab.columns = ['login_colaborador', 'total_spells', 'inicio_segunda', 'fim_sexta']

# Calcular proporções
df_padroes_colab['prop_inicio_segunda'] = df_padroes_colab['inicio_segunda'] / df_padroes_colab['total_spells']
df_padroes_colab['prop_fim_sexta'] = df_padroes_colab['fim_sexta'] / df_padroes_colab['total_spells']

# Flag: > 50% dos spells começam à segunda OU terminam à sexta
df_padroes_colab['flag_weekend_pattern'] = (
    (df_padroes_colab['prop_inicio_segunda'] > 0.5) | 
    (df_padroes_colab['prop_fim_sexta'] > 0.5)
) & (df_padroes_colab['total_spells'] >= 3)  # Mínimo 3 spells para ser significativo

# Adicionar nomes
df_padroes_colab = df_padroes_colab.merge(
    df[['login_colaborador', 'nome_colaborador']].drop_duplicates(),
    on='login_colaborador'
)

n_flagged = df_padroes_colab['flag_weekend_pattern'].sum()
print(f'\n🚩 COLABORADORES COM PADRÃO SUSPEITO: {n_flagged}')
print(f'   (>50% spells começam 2ª OU terminam 6ª, com mínimo 3 spells)')

if n_flagged > 0:
    print(f'\nTop 10:')
    top_weekend = df_padroes_colab[df_padroes_colab['flag_weekend_pattern']].nlargest(10, 'total_spells')[[
        'nome_colaborador', 'total_spells', 'inicio_segunda', 'fim_sexta', 
        'prop_inicio_segunda', 'prop_fim_sexta'
    ]].copy()
    top_weekend['prop_inicio_segunda'] = top_weekend['prop_inicio_segunda'].apply(lambda x: f'{x*100:.1f}%')
    top_weekend['prop_fim_sexta'] = top_weekend['prop_fim_sexta'].apply(lambda x: f'{x*100:.1f}%')
    print(top_weekend.to_string(index=False))

In [ ]:
# 6.2 Padrões de Ponte - Ausências adjacentes a feriados
print('\nAnalisando padrões de ponte (adjacentes a feriados)...')

# NOTA: Precisamos de uma lista de feriados
# Para demonstração, vamos criar feriados típicos portugueses
import pandas as pd
from datetime import datetime

# Feriados fixos comuns (exemplo para um ano)
ano_min = df['Data'].min().year
ano_max = df['Data'].max().year

feriados = []
for ano in range(ano_min, ano_max + 1):
    feriados.extend([
        datetime(ano, 1, 1),   # Ano Novo
        datetime(ano, 4, 25),  # 25 de Abril
        datetime(ano, 5, 1),   # Dia do Trabalhador
        datetime(ano, 6, 10),  # Dia de Portugal
        datetime(ano, 8, 15),  # Assunção
        datetime(ano, 10, 5),  # Implantação da República
        datetime(ano, 11, 1),  # Todos os Santos
        datetime(ano, 12, 1),  # Restauração da Independência
        datetime(ano, 12, 8),  # Imaculada Conceição
        datetime(ano, 12, 25), # Natal
    ])

feriados = pd.to_datetime(feriados)

print(f'   Feriados considerados: {len(feriados)}')

# Identificar spells que começam/terminam adjacentes a feriados
def is_adjacent_to_holiday(date, holidays, tolerance=1):
    """Verifica se data está a ±tolerance dias de um feriado"""
    for holiday in holidays:
        diff = abs((date - holiday).days)
        if diff <= tolerance:
            return True
    return False

df_spells['inicio_adjacente_feriado'] = df_spells['data_inicio'].apply(
    lambda x: is_adjacent_to_holiday(x, feriados)
)

df_spells['fim_adjacente_feriado'] = df_spells['data_fim'].apply(
    lambda x: is_adjacent_to_holiday(x, feriados)
)

n_inicio_adj = df_spells['inicio_adjacente_feriado'].sum()
n_fim_adj = df_spells['fim_adjacente_feriado'].sum()

print(f'\n📊 PADRÕES DE PONTE')
print(f'   Spells que começam adjacentes a feriado: {n_inicio_adj:,} ({n_inicio_adj/len(df_spells)*100:.2f}%)')
print(f'   Spells que terminam adjacentes a feriado: {n_fim_adj:,} ({n_fim_adj/len(df_spells)*100:.2f}%)')

# Por colaborador
df_ponte_colab = df_spells.groupby('login_colaborador').agg({
    'spell_id': 'count',
    'inicio_adjacente_feriado': 'sum',
    'fim_adjacente_feriado': 'sum'
}).reset_index()

df_ponte_colab.columns = ['login_colaborador', 'total_spells', 'inicio_adj_feriado', 'fim_adj_feriado']
df_ponte_colab['prop_ponte'] = (
    (df_ponte_colab['inicio_adj_feriado'] + df_ponte_colab['fim_adj_feriado']) / 
    (df_ponte_colab['total_spells'] * 2)
)

# Flag: > 40% adjacentes a feriados
df_ponte_colab['flag_bridge_pattern'] = (
    (df_ponte_colab['prop_ponte'] > 0.4) & 
    (df_ponte_colab['total_spells'] >= 3)
)

df_ponte_colab = df_ponte_colab.merge(
    df[['login_colaborador', 'nome_colaborador']].drop_duplicates(),
    on='login_colaborador'
)

n_flagged_ponte = df_ponte_colab['flag_bridge_pattern'].sum()
print(f'\n🚩 COLABORADORES COM PADRÃO DE PONTE SUSPEITO: {n_flagged_ponte}')

if n_flagged_ponte > 0:
    print(f'\nTop 10:')
    top_ponte = df_ponte_colab[df_ponte_colab['flag_bridge_pattern']].nlargest(10, 'total_spells')[[
        'nome_colaborador', 'total_spells', 'inicio_adj_feriado', 'fim_adj_feriado'
    ]]
    print(top_ponte.to_string(index=False))

In [ ]:
# 6.3 Outlier Detection - Colaboradores estatisticamente fora do padrão
print('\nDeteção de outliers estatísticos...')

from scipy import stats

# Z-score para várias métricas
df_outliers = df_bradford.copy()

# Calcular Z-scores
df_outliers['z_num_spells'] = stats.zscore(df_outliers['num_spells'])
df_outliers['z_total_dias'] = stats.zscore(df_outliers['total_dias_ausentes'])
df_outliers['z_bradford'] = stats.zscore(df_outliers['bradford_score'])

# Outlier se Z-score > 3 em qualquer métrica
df_outliers['is_outlier'] = (
    (abs(df_outliers['z_num_spells']) > 3) |
    (abs(df_outliers['z_total_dias']) > 3) |
    (abs(df_outliers['z_bradford']) > 3)
)

n_outliers = df_outliers['is_outlier'].sum()
print(f'\n🎯 OUTLIERS DETETADOS: {n_outliers}')
print(f'   ({n_outliers / len(df_outliers) * 100:.2f}% dos colaboradores)')

if n_outliers > 0:
    print(f'\nOutliers (Z-score > 3 em alguma métrica):')
    outliers = df_outliers[df_outliers['is_outlier']][
        ['nome_colaborador', 'num_spells', 'total_dias_ausentes', 'bradford_score',
         'z_num_spells', 'z_total_dias', 'z_bradford']
    ].nlargest(15, 'z_bradford')
    
    for col in ['z_num_spells', 'z_total_dias', 'z_bradford']:
        outliers[col] = outliers[col].apply(lambda x: f'{x:.2f}')
    
    print(outliers.to_string(index=False))

---

## 7. ANÁLISE DE COHORTS (por Data de Ingresso)

**Objetivo**: Investigar se colaboradores mais novos têm taxas de absentismo diferentes.

**Hipóteses a testar**:
- Novos colaboradores podem ter mais ausências (adaptação, problemas iniciais)
- Ou menos ausências (honeymoon period, medo de consequências)

In [ ]:
# 7.1 Análise de Cohorts
print('Analisando cohorts por data de ingresso...')

# NOTA: Precisamos da data de ingresso de cada colaborador
# Se não existir no dataset, criar campo sintético para demonstração
if 'data_ingresso' not in df.columns:
    print('\n⚠️  Campo "data_ingresso" não encontrado no dataset.')
    print('   Criando datas sintéticas para demonstração...')
    
    # Criar datas aleatórias entre 2020 e data_inicio
    np.random.seed(42)
    colaboradores = df['login_colaborador'].unique()
    data_inicio_empresa = pd.to_datetime('2020-01-01')
    data_fim_contratacao = df['Data'].min()
    
    datas_ingresso = pd.DataFrame({
        'login_colaborador': colaboradores,
        'data_ingresso': pd.to_datetime(
            np.random.randint(
                data_inicio_empresa.value // 10**9,
                data_fim_contratacao.value // 10**9,
                size=len(colaboradores)
            ),
            unit='s'
        )
    })
    
    df = df.merge(datas_ingresso, on='login_colaborador', how='left')
    print(f'   ✓ Datas de ingresso criadas para {len(colaboradores):,} colaboradores')

# Calcular tenure (tempo na empresa)
referencia_date = df['Data'].max()
df['tenure_dias'] = (referencia_date - df['data_ingresso']).dt.days

# Categorizar por tenure
df['cohort'] = pd.cut(
    df['tenure_dias'],
    bins=[0, 180, 365, 730, 1095, float('inf')],
    labels=['0-6 meses', '6-12 meses', '1-2 anos', '2-3 anos', '>3 anos']
)

print(f'\nDistribuição de colaboradores por cohort:')
print(df.groupby('cohort')['login_colaborador'].nunique())

# Calcular métricas por cohort
df_cohort = df_bradford.merge(
    df[['login_colaborador', 'data_ingresso', 'tenure_dias', 'cohort']].drop_duplicates(),
    on='login_colaborador',
    how='left'
)

cohort_metrics = df_cohort.groupby('cohort').agg({
    'login_colaborador': 'count',
    'num_spells': 'mean',
    'total_dias_ausentes': 'mean',
    'mean_spell_duration': 'mean',
    'bradford_score': 'mean'
}).reset_index()

cohort_metrics.columns = [
    'Cohort', 'N Colaboradores', 'Média Spells', 'Média Dias Ausentes',
    'Média Duração Spell', 'Média Bradford'
]

print(f'\n📊 MÉTRICAS POR COHORT:')
print(cohort_metrics.to_string(index=False))

---

## 8. CLUSTERING DE PERFIS DE ABSENTISMO

**Objetivo**: Segmentar colaboradores em grupos homogéneos por comportamento de absentismo.

**Features para clustering**:
- Número de spells (frequência)
- Total de dias ausentes
- Duração média de spell
- Bradford score
- Proporção de short-term spells

**Algoritmo**: K-Means (3-5 clusters típicos)

In [ ]:
# 8.1 Preparar dados para clustering
print('Preparando features para clustering...')

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Selecionar features
features_clustering = [
    'num_spells',
    'total_dias_ausentes', 
    'mean_spell_duration',
    'bradford_score',
    'num_short_term_spells'
]

df_cluster = df_bradford[['login_colaborador', 'nome_colaborador'] + features_clustering].copy()

# Normalizar features (importante para K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster[features_clustering])

print(f'\n✓ Features preparadas: {X_scaled.shape}')

# Determinar número ideal de clusters (Elbow Method)
print('\nCalculando Elbow Method (K=2 a K=8)...')
inertias = []
K_range = range(2, 9)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Visualizar Elbow
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(K_range),
    y=inertias,
    mode='lines+markers',
    marker=dict(size=10)
))
fig.update_layout(
    title='Elbow Method - Determinar K Ideal',
    xaxis_title='Número de Clusters (K)',
    yaxis_title='Inertia (Within-Cluster Sum of Squares)',
    height=400
)
fig.show()

# Aplicar K-Means com K=4 (típico para absentismo: baixo/moderado/alto/muito alto)
K_optimal = 4
print(f'\nAplicando K-Means com K={K_optimal}...')

kmeans = KMeans(n_clusters=K_optimal, random_state=42, n_init=10)
df_cluster['cluster'] = kmeans.fit_predict(X_scaled)

print(f'\n✓ Clusters atribuídos')
print(f'\nDistribuição por cluster:')
print(df_cluster['cluster'].value_counts().sort_index())

In [ ]:
# 8.2 Interpretar clusters - Calcular médias por cluster
print('\n=== CARACTERIZAÇÃO DOS CLUSTERS ===')

cluster_profiles = df_cluster.groupby('cluster')[features_clustering].mean()

# Ordenar por bradford_score (do menor ao maior)
cluster_profiles = cluster_profiles.sort_values('bradford_score')

# Renomear clusters de forma interpretável
cluster_mapping = {}
for idx, (cluster_id, row) in enumerate(cluster_profiles.iterrows()):
    if idx == 0:
        cluster_mapping[cluster_id] = '1. BAIXO Absentismo'
    elif idx == 1:
        cluster_mapping[cluster_id] = '2. MODERADO Absentismo'
    elif idx == 2:
        cluster_mapping[cluster_id] = '3. ALTO Absentismo'
    else:
        cluster_mapping[cluster_id] = '4. MUITO ALTO Absentismo'

df_cluster['cluster_nome'] = df_cluster['cluster'].map(cluster_mapping)

# Exibir perfis
print('\nPERFIL MÉDIO POR CLUSTER:\n')
for cluster_id, row in cluster_profiles.iterrows():
    cluster_nome = cluster_mapping[cluster_id]
    n_colab = (df_cluster['cluster'] == cluster_id).sum()
    
    print(f'{cluster_nome} (n={n_colab})')
    print(f'   Nº Spells: {row["num_spells"]:.2f}')
    print(f'   Total Dias Ausentes: {row["total_dias_ausentes"]:.2f}')
    print(f'   Duração Média Spell: {row["mean_spell_duration"]:.2f} dias')
    print(f'   Bradford Score: {row["bradford_score"]:.2f}')
    print(f'   Short-term Spells: {row["num_short_term_spells"]:.2f}')
    print()

# Exportar colaboradores com clusters
df_cluster_export = df_cluster[[
    'nome_colaborador', 'cluster_nome', 'num_spells', 'total_dias_ausentes',
    'mean_spell_duration', 'bradford_score'
]].sort_values(['cluster_nome', 'bradford_score'], ascending=[True, False])

df_cluster_export.to_excel('colaboradores_clusters_absentismo.xlsx', index=False)
print('✓ Clusters exportados para: colaboradores_clusters_absentismo.xlsx')

In [ ]:
# 8.3 Visualizar clusters em 2D (PCA)
print('\nVisualizando clusters em 2D (PCA)...')

from sklearn.decomposition import PCA

# Reduzir para 2 dimensões
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

df_cluster['PC1'] = X_pca[:, 0]
df_cluster['PC2'] = X_pca[:, 1]

print(f'Variância explicada: PC1={pca.explained_variance_ratio_[0]*100:.2f}%, PC2={pca.explained_variance_ratio_[1]*100:.2f}%')

# Scatter plot
fig = go.Figure()

for cluster_nome in sorted(df_cluster['cluster_nome'].unique()):
    df_temp = df_cluster[df_cluster['cluster_nome'] == cluster_nome]
    
    fig.add_trace(go.Scatter(
        x=df_temp['PC1'],
        y=df_temp['PC2'],
        mode='markers',
        name=cluster_nome,
        text=df_temp['nome_colaborador'],
        marker=dict(size=8, opacity=0.7),
        hovertemplate='<b>%{text}</b><br>PC1: %{x:.2f}<br>PC2: %{y:.2f}<extra></extra>'
    ))

fig.update_layout(
    title='Clusters de Absentismo (PCA 2D)',
    xaxis_title=f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)',
    yaxis_title=f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)',
    height=600
)

fig.show()

---

## 9. NETWORK ANALYSIS (Coincidências de Ausências)

**Objetivo**: Detetar padrões de ausências **simultâneas** que possam indicar:
- Eventos locais (gripe, problema na operação)
- Problemas de equipa
- Coincidências suspeitas

**Método**: Identificar dias com múltiplos colaboradores ausentes na mesma operação.

### 9.1 Intuição e Objetivos

**Network Analysis** transforma o problema "quem faltou em que dias" numa **rede (grafo)** onde:
- **Nó** = Colaborador
- **Aresta** = Conexão entre dois colaboradores que faltaram no mesmo dia

**Objetivos**:
1. Descobrir **pares/grupos** que faltam juntos com frequência anómala
2. Identificar **equipas/operações** com alta co-incidência
3. Detetar **eventos pontuais** vs **padrões persistentes**
4. **Priorizar investigações** RH

**Metodologia**:
- Matriz de **co-ocorrência** (quantas vezes A e B faltaram juntos)
- **Testes estatísticos** (hipergeométrico) para filtrar coincidências significativas
- **Community detection** (Louvain) para encontrar clusters
- **Centralidade** para priorizar investigações
- **Visualização interativa** (pyvis)

In [ ]:
# 9.2 Preparar dados: Matriz Bipartida (Colaboradores × Dias)
print('=== NETWORK ANALYSIS: CO-OCORRÊNCIA DE AUSÊNCIAS ===')
print('\nPreparando matriz bipartida...')

# Filtrar apenas ausências (excluir Trabalho Pago, Atraso)
df_ausencias_network = df[
    df['Nivel 1'].isin(['Falta Justificada', 'Falta Injustificada', 'Ausência'])
].copy()

print(f'   Registos de ausência: {len(df_ausencias_network):,}')
print(f'   Colaboradores: {df_ausencias_network["login_colaborador"].nunique():,}')
print(f'   Dias únicos: {df_ausencias_network["Data"].nunique():,}')

# Criar identificador único de dia
df_ausencias_network['date_id'] = df_ausencias_network['Data'].astype(str)

# Lista de colaboradores e dias
colaboradores = sorted(df_ausencias_network['login_colaborador'].unique())
dias = sorted(df_ausencias_network['date_id'].unique())

print(f'\nDimensão da matriz: {len(colaboradores):,} colaboradores × {len(dias):,} dias')
print(f'   Esparsidade: {len(df_ausencias_network) / (len(colaboradores) * len(dias)) * 100:.2f}%')

In [ ]:
# 9.3 Criar Matriz de Co-ocorrência (C = B × B.T)
print('\nCriando matriz de co-ocorrência...')
print('   (Isto pode demorar alguns minutos para datasets grandes)')

from scipy.sparse import csr_matrix, lil_matrix
import numpy as np

# Criar dicionários de índice
colab_to_idx = {c: i for i, c in enumerate(colaboradores)}
date_to_idx = {d: i for i, d in enumerate(dias)}

# Criar matriz bipartida esparsa (colaboradores × dias)
# B[i,j] = 1 se colaborador i esteve ausente no dia j
rows = []
cols = []

for _, row in df_ausencias_network.iterrows():
    colab_idx = colab_to_idx[row['login_colaborador']]
    date_idx = date_to_idx[row['date_id']]
    rows.append(colab_idx)
    cols.append(date_idx)

B = csr_matrix(
    (np.ones(len(rows)), (rows, cols)),
    shape=(len(colaboradores), len(dias)),
    dtype=np.int8
)

print(f'   Matriz B criada: {B.shape}')
print(f'   Memória: {B.data.nbytes / 1024 / 1024:.2f} MB')

# Calcular co-ocorrência: C = B × B.T
# C[i,j] = número de dias em que colaboradores i e j estiveram AMBOS ausentes
print('\n   Calculando C = B × B.T...')
C = B @ B.T

# Converter para array denso (se couber em memória)
# Se dataset muito grande, trabalhar com sparse
if len(colaboradores) < 5000:
    C_dense = C.toarray()
    print(f'   Matriz C: {C_dense.shape} (densa)')
else:
    C_dense = None
    print(f'   Matriz C: {C.shape} (esparsa - dataset grande)')

print('\n✓ Matriz de co-ocorrência criada')

In [ ]:
# 9.4 Teste Estatístico: P-values (Distribuição Hipergeométrica)
print('\nCalculando p-values para coincidências...')

from scipy.stats import hypergeom

# Para cada par (i,j), calcular:
# H0: ausências são aleatórias ao longo de N dias
# N = total de dias
# K = dias ausentes de i (C[i,i])
# M = dias ausentes de j (C[j,j])
# k = observado (C[i,j])
# P(X >= k) sob H0

N_dias = len(dias)

# Dias ausentes por colaborador (diagonal de C)
dias_ausentes = np.diag(C.toarray()) if C_dense is None else np.diag(C_dense)

print(f'   N = {N_dias} dias totais')
print(f'   Média de dias ausentes por colaborador: {dias_ausentes.mean():.2f}')

# Criar lista de pares significativos
# (só calcular para pares com k > 0 para eficiência)
pares_significativos = []

print('\n   Testando pares com co-ocorrências...')

# Threshold mínimo de co-ocorrências para considerar
min_cooccur = 2  # Pelo menos 2 dias juntos

n_pares_testados = 0
if C_dense is not None:
    # Trabalhar com matriz densa
    for i in range(len(colaboradores)):
        if i % 500 == 0 and i > 0:
            print(f'      Processado {i:,}/{len(colaboradores):,} colaboradores...')

        for j in range(i+1, len(colaboradores)):
            k_obs = C_dense[i, j]

            if k_obs >= min_cooccur:
                n_pares_testados += 1

                K_i = dias_ausentes[i]
                M_j = dias_ausentes[j]

                # P-value: P(X >= k_obs) sob hipergeométrica
                # hypergeom(M, n, N) onde M=N_dias, n=K_i, N=M_j
                # Survival function: P(X >= k)
                try:
                    pval = hypergeom.sf(k_obs - 1, N_dias, K_i, M_j)
                except:
                    pval = 1.0  # Se erro, considerar não significativo

                pares_significativos.append({
                    'colaborador_i': colaboradores[i],
                    'colaborador_j': colaboradores[j],
                    'dias_i': K_i,
                    'dias_j': M_j,
                    'cooccur': k_obs,
                    'p_value': pval
                })
else:
    # Trabalhar com matriz esparsa (mais lento mas menos memória)
    C_lil = C.tolil()
    for i in range(len(colaboradores)):
        if i % 500 == 0 and i > 0:
            print(f'      Processado {i:,}/{len(colaboradores):,} colaboradores...')

        for j in range(i+1, len(colaboradores)):
            k_obs = C_lil[i, j]

            if k_obs >= min_cooccur:
                n_pares_testados += 1

                K_i = C_lil[i, i]
                M_j = C_lil[j, j]

                try:
                    pval = hypergeom.sf(k_obs - 1, N_dias, K_i, M_j)
                except:
                    pval = 1.0

                pares_significativos.append({
                    'colaborador_i': colaboradores[i],
                    'colaborador_j': colaboradores[j],
                    'dias_i': K_i,
                    'dias_j': M_j,
                    'cooccur': k_obs,
                    'p_value': pval
                })

df_pares = pd.DataFrame(pares_significativos)

print(f'\n✓ Testados {n_pares_testados:,} pares com ≥{min_cooccur} co-ocorrências')
print(f'   Total de pares potenciais: {len(colaboradores) * (len(colaboradores)-1) // 2:,}')

In [ ]:
# 9.5 Correção FDR (False Discovery Rate)
print('\nAplicando correção FDR...')

from statsmodels.stats.multitest import multipletests

if len(df_pares) > 0:
    # Aplicar Benjamini-Hochberg FDR
    reject, pvals_adj, _, _ = multipletests(
        df_pares['p_value'],
        alpha=0.05,
        method='fdr_bh'
    )

    df_pares['p_adj'] = pvals_adj
    df_pares['significant'] = reject

    n_sig = reject.sum()
    print(f'   Pares significativos (FDR < 0.05): {n_sig:,}')
    print(f'   ({n_sig / len(df_pares) * 100:.2f}% dos pares testados)')

    # Filtrar apenas significativos
    df_pares_sig = df_pares[df_pares['significant']].copy()

    # Ordenar por co-ocorrências (descendente)
    df_pares_sig = df_pares_sig.sort_values('cooccur', ascending=False).reset_index(drop=True)

    print(f'\n📊 TOP 20 PARES COM COINCIDÊNCIAS SIGNIFICATIVAS:')
    print('='*100)

    top20_pares = df_pares_sig.head(20).copy()

    # Adicionar nomes
    top20_pares = top20_pares.merge(
        df[['login_colaborador', 'nome_colaborador']].drop_duplicates(),
        left_on='colaborador_i',
        right_on='login_colaborador',
        how='left'
    ).rename(columns={'nome_colaborador': 'nome_i'}).drop('login_colaborador', axis=1)

    top20_pares = top20_pares.merge(
        df[['login_colaborador', 'nome_colaborador']].drop_duplicates(),
        left_on='colaborador_j',
        right_on='login_colaborador',
        how='left'
    ).rename(columns={'nome_colaborador': 'nome_j'}).drop('login_colaborador', axis=1)

    # Adicionar operação (se disponível)
    if 'operacao' in df.columns:
        top20_pares = top20_pares.merge(
            df[['login_colaborador', 'operacao']].drop_duplicates(),
            left_on='colaborador_i',
            right_on='login_colaborador',
            how='left'
        ).rename(columns={'operacao': 'operacao_i'}).drop('login_colaborador', axis=1)

        top20_pares = top20_pares.merge(
            df[['login_colaborador', 'operacao']].drop_duplicates(),
            left_on='colaborador_j',
            right_on='login_colaborador',
            how='left'
        ).rename(columns={'operacao': 'operacao_j'}).drop('login_colaborador', axis=1)

    # Exibir
    for idx, row in top20_pares.iterrows():
        print(f"\n{idx+1}. {row['nome_i']} + {row['nome_j']}")
        print(f"   Co-ocorrências: {row['cooccur']} dias")
        print(f"   P-value ajustado: {row['p_adj']:.2e}")
        if 'operacao_i' in row:
            print(f"   Operações: {row['operacao_i']} + {row['operacao_j']}")

    # Exportar para Excel
    top20_pares.to_excel('network_top20_pares_significativos.xlsx', index=False)
    print('\n✓ Top 20 pares exportados para: network_top20_pares_significativos.xlsx')

else:
    print('   Nenhum par encontrado')
    df_pares_sig = pd.DataFrame()

In [ ]:
# 9.6 Community Detection (Louvain Algorithm)
print('\nDeteção de comunidades (Louvain)...')

if len(df_pares_sig) > 0:
    import networkx as nx
    from networkx.algorithms import community

    # Criar grafo apenas com arestas significativas
    G = nx.Graph()

    # Adicionar nós
    G.add_nodes_from(colaboradores)

    # Adicionar arestas (ponderadas por co-ocorrências)
    for _, row in df_pares_sig.iterrows():
        G.add_edge(
            row['colaborador_i'],
            row['colaborador_j'],
            weight=row['cooccur'],
            p_adj=row['p_adj']
        )

    print(f'   Grafo criado: {G.number_of_nodes():,} nós, {G.number_of_edges():,} arestas')

    # Aplicar Louvain
    print('   Aplicando algoritmo de Louvain...')
    communities_generator = community.greedy_modularity_communities(G, weight='weight')
    communities_list = list(communities_generator)

    print(f'   Comunidades detetadas: {len(communities_list)}')

    # Criar mapeamento colaborador -> comunidade
    colab_to_community = {}
    for comm_id, comm_members in enumerate(communities_list):
        for member in comm_members:
            colab_to_community[member] = comm_id

    # Estatísticas por comunidade
    print(f'\n📊 COMUNIDADES (clusters de co-ausências):')
    print('='*80)

    for comm_id, comm_members in enumerate(sorted(communities_list, key=len, reverse=True)[:10]):
        print(f'\nComunidade {comm_id + 1}: {len(comm_members)} membros')

        # Nomes (primeiros 5)
        member_names = []
        for member in list(comm_members)[:5]:
            nome = df[df['login_colaborador'] == member]['nome_colaborador'].iloc[0]
            member_names.append(nome)

        print(f'   Membros (sample): {', '.join(member_names)}...')

        # Operações predominantes
        if 'operacao' in df.columns:
            ops = df[df['login_colaborador'].isin(comm_members)]['operacao'].value_counts().head(3)
            print(f'   Operações predominantes:')
            for op, count in ops.items():
                print(f'      - {op}: {count} registos')

    # Salvar comunidades
    df_communities = pd.DataFrame([
        {'login_colaborador': colab, 'community_id': comm_id}
        for colab, comm_id in colab_to_community.items()
    ])

    df_communities = df_communities.merge(
        df[['login_colaborador', 'nome_colaborador']].drop_duplicates(),
        on='login_colaborador'
    )

    df_communities.to_excel('network_comunidades.xlsx', index=False)
    print('\n✓ Comunidades exportadas para: network_comunidades.xlsx')

else:
    print('   Nenhuma aresta significativa - comunidades não podem ser detetadas')
    G = None
    colab_to_community = {}

In [ ]:
# 9.7 Centralidade: Priorizar Investigações
print('\nCalculando métricas de centralidade...')

if G is not None and G.number_of_edges() > 0:
    # Degree centrality (número de conexões)
    degree_cent = nx.degree_centrality(G)

    # Betweenness centrality (ponte entre clusters)
    print('   Calculando betweenness centrality (pode demorar)...')
    betweenness_cent = nx.betweenness_centrality(G, weight='weight')

    # Closeness centrality
    # print('   Calculando closeness centrality...')
    # closeness_cent = nx.closeness_centrality(G, distance='weight')

    # Weighted degree (soma dos pesos)
    weighted_degree = dict(G.degree(weight='weight'))

    # Criar DataFrame
    df_centrality = pd.DataFrame({
        'login_colaborador': list(degree_cent.keys()),
        'degree_centrality': list(degree_cent.values()),
        'betweenness_centrality': list(betweenness_cent.values()),
        # 'closeness_centrality': list(closeness_cent.values()),
        'weighted_degree': [weighted_degree[c] for c in degree_cent.keys()]
    })

    # Adicionar nomes
    df_centrality = df_centrality.merge(
        df[['login_colaborador', 'nome_colaborador']].drop_duplicates(),
        on='login_colaborador'
    )

    # Top 20 por degree
    print(f'\n🎯 TOP 20 COLABORADORES POR CENTRALIDADE (hubs de co-ausência):')
    print('='*100)

    top20_centrality = df_centrality.nlargest(20, 'degree_centrality')

    for idx, row in top20_centrality.iterrows():
        print(f"{row['nome_colaborador']:40s} | Connections: {int(row['weighted_degree']):3d} | "
              f"Degree: {row['degree_centrality']:.4f} | Betweenness: {row['betweenness_centrality']:.4f}")

    # Exportar
    df_centrality.to_excel('network_centralidade.xlsx', index=False)
    print('\n✓ Métricas de centralidade exportadas para: network_centralidade.xlsx')

else:
    print('   Grafo vazio ou sem arestas')
    df_centrality = pd.DataFrame()

In [ ]:
# 9.8 Visualização Interativa (Pyvis)
print('\nCriando visualização interativa da rede...')

if G is not None and G.number_of_edges() > 0:
    try:
        from pyvis.network import Network

        # Limitar a top comunidades e top pares para legibilidade
        # Filtrar apenas top 5 comunidades e top 50 pares
        top_communities = sorted(communities_list, key=len, reverse=True)[:5]
        top_community_ids = [i for i, comm in enumerate(communities_list) if comm in top_communities]

        # Nós da top comunidades
        top_nodes = set()
        for comm in top_communities:
            top_nodes.update(comm)

        # Criar subgrafo
        G_sub = G.subgraph(top_nodes).copy()

        print(f'   Visualizando subgrafo: {G_sub.number_of_nodes()} nós, {G_sub.number_of_edges()} arestas')

        # Criar network pyvis
        net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

        # Configurações
        net.barnes_hut(gravity=-5000, central_gravity=0.3, spring_length=100)

        # Cores por comunidade
        colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231',
                  '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe']

        # Adicionar nós
        for node in G_sub.nodes():
            # Nome
            nome = df[df['login_colaborador'] == node]['nome_colaborador'].iloc[0] if len(df[df['login_colaborador'] == node]) > 0 else node

            # Comunidade
            comm_id = colab_to_community.get(node, -1)
            color = colors[comm_id % len(colors)] if comm_id >= 0 else '#cccccc'

            # Tamanho proporcional ao degree
            degree = G_sub.degree(node, weight='weight')
            size = 10 + degree * 2

            net.add_node(
                node,
                label=nome[:20],  # Limitar label
                title=f"{nome}\nComunidade: {comm_id+1}\nDegree: {degree}",
                color=color,
                size=size
            )

        # Adicionar arestas
        for edge in G_sub.edges(data=True):
            net.add_edge(
                edge[0],
                edge[1],
                value=edge[2]['weight'],  # Espessura proporcional ao peso
                title=f"{edge[2]['weight']} co-ocorrências"
            )

        # Salvar
        net.save_graph('network_visualization.html')
        print('\n✓ Visualização interativa criada: network_visualization.html')
        print('   (Abrir no browser para explorar)')

    except ImportError:
        print('\n⚠️  pyvis não instalado. Instalando...')
        import subprocess
        subprocess.run(['pip', 'install', 'pyvis', '-q'])
        print('   Por favor, executar esta célula novamente')

else:
    print('   Grafo vazio - visualização não criada')

In [ ]:
# 9.9 Dinâmica Temporal: Evolução dos Pares Significativos
print('\nAnalisando dinâmica temporal da rede...')

if len(df_pares_sig) > 0:
    # Para cada par significativo, encontrar QUANDO aconteceram as co-ausências

    # Pegar top 10 pares
    top_pares = df_pares_sig.head(10)

    # Para cada par, buscar datas
    timeline_data = []

    for _, pair in top_pares.iterrows():
        colab_i = pair['colaborador_i']
        colab_j = pair['colaborador_j']

        # Encontrar dias em que AMBOS faltaram
        dias_i = set(df_ausencias_network[df_ausencias_network['login_colaborador'] == colab_i]['Data'])
        dias_j = set(df_ausencias_network[df_ausencias_network['login_colaborador'] == colab_j]['Data'])

        dias_comuns = sorted(dias_i & dias_j)

        # Agrupar por mês
        for data in dias_comuns:
            mes = data.to_period('M').to_timestamp()
            timeline_data.append({
                'par': f"{pair['nome_i'][:15]}...\n{pair['nome_j'][:15]}...",
                'mes': mes,
                'cooccur_count': 1
            })

    if timeline_data:
        df_timeline = pd.DataFrame(timeline_data)

        # Agregar por par e mês
        df_timeline_agg = df_timeline.groupby(['par', 'mes'])['cooccur_count'].sum().reset_index()

        # Plot
        fig = px.line(
            df_timeline_agg,
            x='mes',
            y='cooccur_count',
            color='par',
            title='Evolução Temporal: Co-ocorrências dos Top 10 Pares',
            labels={'mes': 'Mês', 'cooccur_count': 'Nº Co-ocorrências', 'par': 'Par'},
            height=600
        )

        fig.update_traces(mode='lines+markers')
        fig.update_layout(hovermode='x unified')

        fig.show()

        print('✓ Timeline criada')
    else:
        print('   Sem dados suficientes para timeline')
else:
    print('   Nenhum par significativo para análise temporal')

---

## 10. EVENT DETECTION & ANOMALY DETECTION

**Objetivo**: Monitorar evolução temporal e detetar **mudanças de padrão** (changepoints).

**Métodos**:
- **U-Chart**: Control chart para taxa por unidade de tempo
- **Rolling statistics**: Média móvel e desvio padrão
- **Anomaly detection**: Períodos estatisticamente anómalos

In [ ]:
# 10.1 U-Chart - Taxa de ausências por semana
print('Criando U-Chart (control chart)...')

# Agrupar por semana
df_ausencias['semana'] = df_ausencias['Data'].dt.to_period('W').astype(str)

ausencias_semana = df_ausencias.groupby('semana').size().reset_index(name='num_ausencias')
ausencias_semana['semana_dt'] = pd.to_datetime(ausencias_semana['semana'].str.split('/').str[0])

# Calcular colaboradores ativos por semana (para normalizar)
colab_semana = df.groupby(df['Data'].dt.to_period('W').astype(str))['login_colaborador'].nunique()
ausencias_semana['colaboradores_ativos'] = ausencias_semana['semana'].map(colab_semana)

# Taxa de ausências por colaborador
ausencias_semana['taxa_ausencia_colab'] = (
    ausencias_semana['num_ausencias'] / ausencias_semana['colaboradores_ativos']
)

# Limites de controle (U-chart)
# UCL/LCL = mean ± 3*sqrt(mean/n)
mean_taxa = ausencias_semana['taxa_ausencia_colab'].mean()
ausencias_semana['ucl'] = mean_taxa + 3 * np.sqrt(mean_taxa / ausencias_semana['colaboradores_ativos'])
ausencias_semana['lcl'] = mean_taxa - 3 * np.sqrt(mean_taxa / ausencias_semana['colaboradores_ativos'])
ausencias_semana['lcl'] = ausencias_semana['lcl'].clip(lower=0)

# Identificar semanas fora de controle
ausencias_semana['out_of_control'] = (
    (ausencias_semana['taxa_ausencia_colab'] > ausencias_semana['ucl']) |
    (ausencias_semana['taxa_ausencia_colab'] < ausencias_semana['lcl'])
)

n_out = ausencias_semana['out_of_control'].sum()
print(f'\n📊 U-CHART')
print(f'   Semanas analisadas: {len(ausencias_semana)}')
print(f'   Taxa média: {mean_taxa:.3f} ausências/colaborador/semana')
print(f'   Semanas FORA DE CONTROLE: {n_out} ({n_out/len(ausencias_semana)*100:.2f}%)')

# Visualizar U-Chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ausencias_semana['semana_dt'],
    y=ausencias_semana['taxa_ausencia_colab'],
    mode='lines+markers',
    name='Taxa Observada',
    line=dict(color='blue'),
    marker=dict(
        size=6,
        color=ausencias_semana['out_of_control'].map({True: 'red', False: 'blue'})
    )
))

fig.add_trace(go.Scatter(
    x=ausencias_semana['semana_dt'],
    y=[mean_taxa] * len(ausencias_semana),
    mode='lines',
    name='Média',
    line=dict(color='green', dash='dash')
))

fig.add_trace(go.Scatter(
    x=ausencias_semana['semana_dt'],
    y=ausencias_semana['ucl'],
    mode='lines',
    name='UCL (Upper Control Limit)',
    line=dict(color='red', dash='dot')
))

fig.add_trace(go.Scatter(
    x=ausencias_semana['semana_dt'],
    y=ausencias_semana['lcl'],
    mode='lines',
    name='LCL (Lower Control Limit)',
    line=dict(color='red', dash='dot'),
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.1)'
))

fig.update_layout(
    title='U-Chart: Taxa de Ausências por Semana (Control Chart)',
    xaxis_title='Semana',
    yaxis_title='Ausências / Colaborador',
    height=500,
    hovermode='x unified'
)

fig.show()

if n_out > 0:
    print(f'\nSemanas fora de controle:')
    print(ausencias_semana[ausencias_semana['out_of_control']][
        ['semana', 'num_ausencias', 'colaboradores_ativos', 'taxa_ausencia_colab', 'ucl']
    ].to_string(index=False))

---

## 11. VISUALIZAÇÕES AVANÇADAS

### 11.1 Calendar Heatmap - Visualizar dias críticos

In [ ]:
# 11.1 Calendar Heatmap por Operação
print('Criando calendar heatmap...')

if 'operacao' in df_ausencias.columns:
    # Selecionar top 3 operações por número de ausências
    top_operacoes = df_ausencias['operacao'].value_counts().head(3).index.tolist()
    
    for operacao in top_operacoes:
        print(f'\nCriando heatmap para: {operacao}')
        
        df_op = df_ausencias[df_ausencias['operacao'] == operacao].copy()
        
        # Contar ausências por dia
        ausencias_dia = df_op.groupby('Data').size().reset_index(name='num_ausencias')
        
        # Adicionar features para heatmap
        ausencias_dia['ano'] = ausencias_dia['Data'].dt.year
        ausencias_dia['semana'] = ausencias_dia['Data'].dt.isocalendar().week
        ausencias_dia['dia_semana'] = ausencias_dia['Data'].dt.dayofweek
        
        # Pivot para heatmap: semanas x dias da semana
        pivot = ausencias_dia.pivot_table(
            index='semana',
            columns='dia_semana',
            values='num_ausencias',
            aggfunc='sum'
        ).fillna(0)
        
        # Criar heatmap
        fig = go.Figure(data=go.Heatmap(
            z=pivot.values,
            x=['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo'],
            y=pivot.index,
            colorscale='Reds',
            hoverongaps=False,
            hovertemplate='Semana %{y}<br>%{x}<br>Ausências: %{z}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'Calendar Heatmap: Ausências - {operacao}',
            xaxis_title='Dia da Semana',
            yaxis_title='Semana do Ano',
            height=600
        )
        
        fig.show()
else:
    print('\n⚠️  Campo "operacao" não encontrado')

---

## 12. SÍNTESE EXECUTIVA E AÇÕES RECOMENDADAS

**Esta secção consolida os principais insights e recomendações para ação.**

In [ ]:
# 12.1 Dashboard Executivo
print('='*100)
print('SÍNTESE EXECUTIVA - ANÁLISE DE ABSENTISMO')
print('='*100)

print(f'\n🎯 KPIs PRINCIPAIS')
print(f'   Taxa de Absentismo Global: {taxa_absentismo_global:.2f}%')
print(f'   Lost Time Rate: {lost_time_rate:.2f} dias/colaborador')
print(f'   Frequency Rate: {frequency_rate:.2f} spells/colaborador')
print(f'   Mean Spell Duration: {mean_spell_duration:.2f} dias')

print(f'\n🚨 ALERTAS CRÍTICOS')
print(f'   Colaboradores Bradford >500 (ação disciplinar): {(df_bradford["bradford_score"] > 500).sum()}')
print(f'   Colaboradores Bradford >900 (preocupação séria): {(df_bradford["bradford_score"] > 900).sum()}')

if 'df_padroes_colab' in locals():
    print(f'   Padrão Segunda/Sexta suspeito: {df_padroes_colab["flag_weekend_pattern"].sum()} colaboradores')

if 'df_ponte_colab' in locals():
    print(f'   Padrão de Ponte suspeito: {df_ponte_colab["flag_bridge_pattern"].sum()} colaboradores')

print(f'   Outliers estatísticos: {n_outliers} colaboradores')

if 'df_surtos' in locals():
    print(f'   Dias com surto de ausências: {len(df_surtos)} dias')

if 'ausencias_semana' in locals():
    print(f'   Semanas fora de controle (U-Chart): {n_out} semanas')

print(f'\n📊 PERFIS DE ABSENTISMO (Clusters)')
if 'df_cluster' in locals():
    for cluster in sorted(df_cluster['cluster_nome'].unique()):
        n = (df_cluster['cluster_nome'] == cluster).sum()
        pct = n / len(df_cluster) * 100
        print(f'   {cluster}: {n} colaboradores ({pct:.1f}%)')

print(f'\n💡 AÇÕES RECOMENDADAS')
print(f'\n1. IMEDIATAS (próximas 2 semanas):')
print(f'   - Conversa com Top 20 Bradford Factor (>= posição de aviso escrito)')
print(f'   - Investigar {len(df_surtos) if "df_surtos" in locals() else 0} dias de surto identificados')
print(f'   - Rever casos de padrão segunda/sexta e ponte')

print(f'\n2. CURTO PRAZO (próximo mês):')
print(f'   - Implementar monitorização contínua (U-Chart semanal)')
print(f'   - Definir política de follow-up por cluster')
print(f'   - Análise detalhada das {n_out if "ausencias_semana" in locals() else 0} semanas fora de controle')

print(f'\n3. MÉDIO PRAZO (próximos 3 meses):')
print(f'   - Programa de engagement para clusters alto/muito alto')
print(f'   - Investigar causas raiz por operação')
print(f'   - Implementar sistema de early warning (Bradford + padrões)')

print(f'\n' + '='*100)

---

## 📝 NOTAS FINAIS

**Ficheiros Gerados**:
- `incompatibilidades_encontradas_v2.xlsx` - Casos de dados inconsistentes removidos
- `matriz_compatibilidade_nivel2_v2.xlsx` - Regras de compatibilidade aplicadas
- `bradford_factor_top20.xlsx` - Colaboradores prioritários para RH
- `colaboradores_clusters_absentismo.xlsx` - Segmentação completa
- `dias_surto_ausencias.xlsx` - Dias com ausências anómalas

**Próximos Passos**:
1. Validar resultados com RH
2. Criar dashboard interativo (Power BI / Tableau)
3. Implementar monitorização contínua
4. Desenvolver ferramenta Excel para consulta (Parte 2)

**Referências**:
- Bradford Factor: Wikipedia, Call Centre Helper
- Spell Analysis: Fitzgerald Human Resources
- HR Analytics: AIHR (Academy to Innovate HR)